In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pandas.core.common import flatten

In [114]:
ts = pd.read_csv("/Users/Isaac/Documents/GitHub/W210-Capstone/models/Time Series/PredictionResults/GDPwSPFPreds.csv")
ts2 = pd.read_csv("/Users/Isaac/Documents/GitHub/W210-Capstone/models/Time Series/PredictionResults/GDPEvalandPreds.csv")

In [115]:
#df = pd.read_csv("../data/CleanData/Ensemble_inputs.csv")
df2 = pd.read_csv("/Users/Isaac/Downloads/model_preds.csv")
df = pd.read_csv("/Users/Isaac/Downloads/train_preds.csv")

In [116]:
#dft = pd.read_csv("../data/TestData/Ensemble_inputs_test.csv")
dft = pd.read_csv("/Users/Isaac/Downloads/test_preds.csv")

In [117]:
#dfp = pd.read_csv("../data/PredictionData/Ensemble_inputs_predict.csv")
dfp = pd.read_csv("/Users/Isaac/Downloads/future_preds.csv")

In [118]:
dfp = dfp[dfp["Year"] < 2023]
dfp

,Unnamed: 0,SPF,Model Pred,Year
0,23259,1.9500,2.553776,2020
1,23264,0.9000,1.327014,2020
2,23265,2.2300,2.285147,2020
3,23267,1.7300,2.170181,2020
4,23269,3.0700,2.629726,2020
5,23270,2.0400,2.430910,2020
6,23271,1.5800,2.515279,2020
7,23272,2.0500,2.216792,2020
8,23273,1.0000,2.485052,2020
9,23274,1.5500,2.321415,2020


In [119]:
df2

,Year,Actual,SPF,Model Val,Model Test
0,1992,3.52244,1.792117,2.283792,2.244451
1,1993,2.75179,2.829303,2.810716,2.806247
2,1994,4.02879,3.197719,3.086463,3.042563
3,1995,2.68421,2.827097,3.071939,3.069233
4,1996,3.77257,2.248873,3.021837,3.086144
5,1997,4.44717,2.704064,3.337892,3.445742
6,1998,4.48139,2.624405,3.268756,3.347521
7,1999,4.79450,3.012362,3.402202,3.426237
8,2000,4.07716,3.491066,3.377662,3.459130
9,2001,0.95434,2.383571,2.222692,2.361134


In [120]:
df

,Unnamed: 0,Actual,SPF,Model Val,Year
0,6272,-2.59989,2.577223,-0.956356,2009
1,16879,2.28778,2.100000,2.404486,2014
2,13310,2.28069,2.450000,2.274789,2012
3,15714,1.66747,2.900000,1.977489,2016
4,15088,2.70637,2.800000,2.758521,2015
5,4755,0.12219,3.300000,1.119432,2008
6,2114,4.07716,2.200000,2.770534,2000
7,514,4.02879,2.800000,3.558559,1994
8,3891,3.48322,3.750000,3.641937,2005
9,1780,4.48139,3.200000,3.778984,1998


In [150]:
ts["year"] = ts["year"].astype(int)
#ts = ts.drop(columns = ["Unnamed: 0", "actual", "pred_average"])
ts
# 2021 : 
# 2022 : 
ts3 = ts
ts3.iloc[16,0] = 2.284382
ts3.iloc[16,1] = 2021
ts3 = ts3.append({"ts": 2.152472, "year": 2022}, ignore_index=True)
ts3

,ts,year
0,4.365155,2005.0
1,3.824596,2006.0
2,1.741411,2007.0
3,0.017238,2008.0
4,-4.405738,2009.0
5,2.331425,2010.0
6,1.780707,2011.0
7,2.244782,2012.0
8,1.741338,2013.0
9,1.525327,2014.0


In [151]:
dat = df.merge(ts, how="left", left_on='Year', right_on='year')
dat2 = dft.merge(ts, how="left", left_on='Year', right_on='year')
dat3 = dfp.merge(ts3, how="left", left_on='Year', right_on='year')

In [152]:
dat3 = dat3.drop(columns=["Unnamed: 0"])

dat3

,SPF,Model Pred,Year,ts,year
0,1.9500,2.553776,2020,-0.654795,2020.0
1,0.9000,1.327014,2020,-0.654795,2020.0
2,2.2300,2.285147,2020,-0.654795,2020.0
3,1.7300,2.170181,2020,-0.654795,2020.0
4,3.0700,2.629726,2020,-0.654795,2020.0
5,2.0400,2.430910,2020,-0.654795,2020.0
6,1.5800,2.515279,2020,-0.654795,2020.0
7,2.0500,2.216792,2020,-0.654795,2020.0
8,1.0000,2.485052,2020,-0.654795,2020.0
9,1.5500,2.321415,2020,-0.654795,2020.0


In [153]:
#dat = dat.drop(columns = ["Unnamed: 0", "actual", "pred_average", "year"])
#dat = dat.iloc[13:, ]
dat = dat.drop(columns=["Unnamed: 0"])
dat

,Actual,SPF,Model Val,Year,ts,year
0,-2.59989,2.577223,-0.956356,2009,-4.405738,2009.0
1,2.28778,2.100000,2.404486,2014,1.525327,2014.0
2,2.28069,2.450000,2.274789,2012,2.244782,2012.0
3,1.66747,2.900000,1.977489,2016,2.086048,2016.0
4,2.70637,2.800000,2.758521,2015,2.338382,2015.0
5,0.12219,3.300000,1.119432,2008,0.017238,2008.0
6,4.07716,2.200000,2.770534,2000,NaN,NaN
7,4.02879,2.800000,3.558559,1994,NaN,NaN
8,3.48322,3.750000,3.641937,2005,4.365155,2005.0
9,4.48139,3.200000,3.778984,1998,NaN,NaN


In [154]:
dat2 = dat2.drop(columns=["Unnamed: 0"])
dat2

,Actual,SPF,Model Test,Year,ts,year
0,2.28069,2.280000,1.901683,2012,2.244782,2012.0
1,2.28887,2.400000,2.342085,2019,2.350946,2019.0
2,2.70637,2.500000,2.733985,2015,2.338382,2015.0
3,2.91886,2.500000,2.679203,2018,2.696346,2018.0
4,2.79621,2.800000,2.128564,2003,NaN,NaN
5,2.01051,2.450000,2.375712,2007,1.741411,2007.0
6,1.54990,2.100000,2.027075,2011,1.780707,2011.0
7,0.95434,2.730000,2.844062,2001,NaN,NaN
8,-3.40459,1.700000,-0.457095,2020,-0.654795,2020.0
9,2.70886,2.000000,1.839226,2010,2.331425,2010.0


In [155]:
dat = dat.dropna()
gX = dat[["Model Val", "ts"]]
gX#, "Linear Regression RealGDP", "KNeigborsRegressor RealGDP"]]
gY = dat[["Actual"]]

In [156]:
reg = LinearRegression().fit(gX, gY)
print(reg.score(gX, gY))

pred = reg.predict(gX)
actual = gY

np.sqrt(mean_squared_error(pred, actual))

0.940967489890382


0.30162758593787753

In [157]:
reg.coef_

array([[0.45484955, 0.4485225 ]])

In [158]:
dat2 = dat2.dropna()

gX = dat2[["Model Test", "ts"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
gY = dat2[["Actual"]]

gX = gX.dropna()
gY = gY.dropna()

print(reg.score(gX, gY))

testpred = reg.predict(gX)
actual = gY

print(np.sqrt(mean_squared_error(testpred, actual)))
print(np.sqrt(mean_squared_error(dat2["SPF"], actual)))

0.6981859907683472
0.9736612439197876
1.589616950604771


In [159]:
dat3 = dat3.dropna()

gX = dat3[["Model Pred", "ts"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
#gY = dat2[["Actual"]]

gX = gX.dropna()
#gY = gY.dropna()

#print(reg.score(gX, gY))

testpred = reg.predict(gX)
#actual = gY

#print(np.sqrt(mean_squared_error(testpred, actual)))
#print(np.sqrt(mean_squared_error(dat2["SPF"], actual)))
dat3["predictions"] = testpred

testdatamaybe2 = dat3.groupby("Year").agg({"predictions": np.mean, "SPF": np.mean})
tosave = testdatamaybe2.iloc[1:, ]
tosave.to_csv("/Users/Isaac/Downloads/EnsembleFuturePredictions.csv")

In [97]:
dat2["predictions"] = testpred
dat2

,Actual,SPF,Model Test,Year,ts,year,predictions
0,2.28069,2.280,1.901683,2012,2.244782,2012.0,2.085390
1,2.28887,2.400,2.342085,2019,2.350946,2019.0,2.333323
2,2.70637,2.500,2.733985,2015,2.338382,2015.0,2.505943
3,2.91886,2.500,2.679203,2018,2.696346,2018.0,2.641581
5,2.01051,2.450,2.375712,2007,1.741411,2007.0,2.075228
6,1.54990,2.100,2.027075,2011,1.780707,2011.0,1.934276
8,-3.40459,1.700,-0.457095,2020,-0.654795,2020.0,-0.288025
9,2.70886,2.000,1.839226,2010,2.331425,2010.0,2.095842
10,-3.40459,-1.850,-2.921770,2020,-0.654795,2020.0,-1.409082
12,0.12219,1.400,0.695691,2008,0.017238,2008.0,0.537741


In [130]:
testdatamaybe = dat2.groupby("Year").agg({"predictions": np.mean, "SPF": np.mean, "Actual": np.mean})
testdatamaybe
testdatamaybe.to_csv("/Users/Isaac/Downloads/EnsemblePredictions.csv")

In [108]:
print(np.sqrt(mean_squared_error(testdatamaybe["predictions"], testdatamaybe["Actual"])))
print(np.sqrt(mean_squared_error(testdatamaybe["SPF"], testdatamaybe["Actual"])))

0.8780617952186635
1.4076136866686604


In [ ]:
gX = dfp[["XGBoost RealGDP", "RandomForest RealGDP"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]

gX = gX.dropna()

testpredg = reg.predict(gX)

In [22]:
df

,Year,Actual,SPF,Model Val,Model Test
0,1992,3.52244,1.792117,2.283792,2.244451
1,1993,2.75179,2.829303,2.810716,2.806247
2,1994,4.02879,3.197719,3.086463,3.042563
3,1995,2.68421,2.827097,3.071939,3.069233
4,1996,3.77257,2.248873,3.021837,3.086144
5,1997,4.44717,2.704064,3.337892,3.445742
6,1998,4.48139,2.624405,3.268756,3.347521
7,1999,4.79450,3.012362,3.402202,3.426237
8,2000,4.07716,3.491066,3.377662,3.459130
9,2001,0.95434,2.383571,2.222692,2.361134


In [15]:
df.head()

,Year,Actual,SPF,Model Val,Model Test
0,1992,3.52244,1.792117,2.283792,2.244451
1,1993,2.75179,2.829303,2.810716,2.806247
2,1994,4.02879,3.197719,3.086463,3.042563
3,1995,2.68421,2.827097,3.071939,3.069233
4,1996,3.77257,2.248873,3.021837,3.086144


In [161]:
dfp.head()

,Unemployment,Unemployment actual,RealGDP,RealGDP actual,Core CPI,Core CPI actual,Core PCE,Core PCE actual,XGBoost Unemployment,XGBoost RealGDP,XGBoost Core CPI,XGBoost Core PCE,RandomForest Unemployment,RandomForest RealGDP,RandomForest Core CPI,RandomForest Core PCE
0,Unemployment,Unknown,RealGDP,Unknown,Core CPI,Unknown,Core PCE,Unknown,7.393668,-0.705191,1.713325,1.378743,7.335446,-0.109095,1.766475,1.376086
1,Unemployment,Unknown,RealGDP,Unknown,Core CPI,Unknown,Core PCE,Unknown,7.910341,-1.261568,1.739907,1.388766,7.871280,-2.209049,1.800801,1.403231
2,Unemployment,Unknown,RealGDP,Unknown,Core CPI,Unknown,Core PCE,Unknown,7.739791,-0.793215,1.737717,1.378743,7.665833,0.196510,1.769844,1.395988
3,Unemployment,Unknown,RealGDP,Unknown,Core CPI,Unknown,Core PCE,Unknown,6.362326,-0.388852,1.740747,1.373877,7.729048,-0.044055,1.796687,1.357659
4,Unemployment,Unknown,RealGDP,Unknown,Core CPI,Unknown,Core PCE,Unknown,7.531527,-0.857580,1.720671,1.378743,6.600268,-0.286090,1.730217,1.415595


# Data Prep Work

In [162]:
uX = df[["XGBoost Unemployment", "RandomForest Unemployment"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
uY = df[["Unemployment actual"]]

uX = uX.dropna()
uY = uY.dropna()

In [163]:
gX = df[["XGBoost RealGDP", "RandomForest RealGDP"]]#, "Linear Regression RealGDP", "KNeigborsRegressor RealGDP"]]
gY = df[["RealGDP actual"]]

In [164]:
cX = df[["XGBoost Core CPI", "RandomForest Core CPI"]]#, "Linear Regression Core CPI", "KNeigborsRegressor Core CPI"]]
cY = df[["Core CPI actual"]]

cX = cX.dropna()
cY = cY.dropna()

In [165]:
pX = df[["XGBoost Core PCE", "RandomForest Core PCE"]]#, "Linear Regression Core PCE", "KNeigborsRegressor Core PCE"]]
pY = df[["Core PCE actual"]]

pX = pX.dropna()
pY = pY.dropna()

# Train the Ensemble Weights

In [166]:
reg = LinearRegression().fit(gX, gY)
print(reg.score(gX, gY))

pred = reg.predict(gX)
actual = gY

np.sqrt(mean_squared_error(pred, actual))

0.7255517468974357


0.9605828153882446

In [167]:
reg.coef_

array([[0.7513008 , 0.23820811]])

In [168]:
gX = dft[["XGBoost RealGDP", "RandomForest RealGDP"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
gY = dft[["RealGDP actual"]]

gX = gX.dropna()
gY = gY.dropna()

print(reg.score(gX, gY))

testpred = reg.predict(gX)
actual = gY

np.sqrt(mean_squared_error(testpred, actual))

0.7166459190386019


0.9463284892883185

In [169]:
gX = dfp[["XGBoost RealGDP", "RandomForest RealGDP"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]

gX = gX.dropna()

testpredg = reg.predict(gX)

In [170]:
reg = LinearRegression().fit(uX, uY)
print(reg.score(uX, uY))

pred = reg.predict(uX)
actual = uY

np.sqrt(mean_squared_error(pred, actual))

0.7925541511778704


0.8812050404877251

In [171]:
reg.coef_

array([[0.89907881, 0.1030051 ]])

In [172]:
uX = dft[["XGBoost Unemployment", "RandomForest Unemployment"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
uY = dft[["Unemployment actual"]]

uX = uX.dropna()
uY = uY.dropna()

print(reg.score(uX, uY))

testpred = reg.predict(uX)
actual = uY

np.sqrt(mean_squared_error(testpred, actual))

0.7724688587635861


0.9139474764257247

In [173]:
uX = dfp[["XGBoost Unemployment", "RandomForest Unemployment"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]

uX = uX.dropna()

testpredu = reg.predict(uX)

In [174]:
reg = LinearRegression().fit(cX, cY)
print(reg.score(cX, cY))

pred = reg.predict(cX)
actual = cY

np.sqrt(mean_squared_error(pred, actual))

0.47296388930613054


0.2943635989333608

In [175]:
reg.coef_

array([[0.93280043, 0.14751195]])

In [176]:
cX = dft[["XGBoost Core CPI", "RandomForest Core CPI"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
cY = dft[["Core CPI actual"]]

cX = cX.dropna()
cY = cY.dropna()

print(reg.score(cX, cY))

testpred = reg.predict(cX)
actual = cY

np.sqrt(mean_squared_error(testpred, actual))

0.45839613833950144


0.30252214686701157

In [177]:
cX = dfp[["XGBoost Core CPI", "RandomForest Core CPI"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]

cX = cX.dropna()

testpredc = reg.predict(cX)

In [178]:
reg = LinearRegression().fit(pX, pY)
print(reg.score(pX, pY))

pred = reg.predict(pX)
actual = pY

np.sqrt(mean_squared_error(pred, actual))

0.5146228020778377


0.21094134019330718

In [179]:
reg.coef_

array([[1.05239414, 0.01996658]])

In [180]:
pX = dft[["XGBoost Core PCE", "RandomForest Core PCE"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
pY = dft[["Core PCE actual"]]

pX = pX.dropna()
pY = pY.dropna()

print(reg.score(pX, pY))

testpred = reg.predict(pX)
actual = pY

np.sqrt(mean_squared_error(testpred, actual))

0.4645981357349809


0.22088178791279664

In [181]:
pX = dfp[["XGBoost Core PCE", "RandomForest Core PCE"]]#, "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]

pX = pX.dropna()

testpredp = reg.predict(pX)

# Do the Aggregate Predictions

In [183]:
# Get predictions set
dftt = pd.read_csv("../data/PredictionData/predict_set.csv")
dftt = dftt.drop("actual", axis=1)
#dftt.groupby("FORECASTER ID")
#len(dftt[dftt["INDICATOR"] == "RealGDP"].drop("actual", axis=1).dropna())
#dftt["INDICATOR"].unique()
dgdp = dftt[dftt["INDICATOR"] == "RealGDP"]
dgdp = dgdp.dropna()

dune = dftt[dftt["INDICATOR"] == "Unemployment"]
dune = dune.dropna()

dpce = dftt[dftt["INDICATOR"] == "Core PCE"]
dpce = dpce.dropna()

dcpi = dftt[dftt["INDICATOR"] == "Core CPI"]
dcpi = dcpi.dropna()

In [184]:
# Create Prediction Columns
dgdp["Preds"] = list(flatten(testpredg))
dune["Preds"] = list(flatten(testpredu))
dpce["Preds"] = list(flatten(testpredp))
dcpi["Preds"] = list(flatten(testpredc))

# Merge into prediction dataframe
dat = pd.concat([dgdp, dune, dpce, dcpi])
dat.head()

,YEAR FORECAST MADE,YEAR BEING FORECAST,FORECASTER_CONF,INDICATOR,pred_average,pred_var,banana,beef,bread,chicken,eggs,electricity,flour,iceCream,unleadedGasoline,Preds
1,2019,2022,0.288582,RealGDP,1.80,1.0100,0.576,3.761333,1.272333,1.475667,1.551667,0.135333,0.442333,4.894667,2.402,-0.546182
6,2019,2022,0.335406,RealGDP,1.19,0.9139,0.576,3.761333,1.272333,1.475667,1.551667,0.135333,0.442333,4.894667,2.402,-1.464415
7,2019,2022,0.306127,RealGDP,1.84,0.9044,0.576,3.761333,1.272333,1.475667,1.551667,0.135333,0.442333,4.894667,2.402,-0.539517
9,2019,2022,0.256955,RealGDP,2.24,1.6524,0.576,3.761333,1.272333,1.475667,1.551667,0.135333,0.442333,4.894667,2.402,-0.293024
10,2019,2022,0.256112,RealGDP,2.32,1.7476,0.576,3.761333,1.272333,1.475667,1.551667,0.135333,0.442333,4.894667,2.402,-0.702834


In [185]:
frame = pd.DataFrame({ "RealGDP Prediction": dat[dat["INDICATOR"]=="RealGDP"]["Preds"], "Year": dat[dat["INDICATOR"]=="RealGDP"]["YEAR BEING FORECAST"]})
groupedgdp = frame.groupby("Year").mean()
groupedgdp

,RealGDP Prediction
Year,
2022,-0.579815
2023,-0.654251
2024,-0.761541


In [186]:
frame = pd.DataFrame({ "Unemployment Prediction": dat[dat["INDICATOR"]=="Unemployment"]["Preds"], "Year": dat[dat["INDICATOR"]=="Unemployment"]["YEAR BEING FORECAST"]})
groupedune = frame.groupby("Year").mean()
groupedune

,Unemployment Prediction
Year,
2022,7.129896
2023,6.973807
2024,7.014714


In [187]:
frame = pd.DataFrame({ "Core PCE Prediction": dat[dat["INDICATOR"]=="Core PCE"]["Preds"], "Year": dat[dat["INDICATOR"]=="Core PCE"]["YEAR BEING FORECAST"]})
groupedpce = frame.groupby("Year").mean()
groupedpce

,Core PCE Prediction
Year,
2022,1.452295


In [188]:
frame = pd.DataFrame({ "Core CPI Prediction": dat[dat["INDICATOR"]=="Core CPI"]["Preds"], "Year": dat[dat["INDICATOR"]=="Core CPI"]["YEAR BEING FORECAST"]})
groupedcpi = frame.groupby("Year").mean()
groupedcpi

,Core CPI Prediction
Year,
2022,1.782852


# Use Gridsearch Results

In [72]:
uX = dfs[["XGBoost Unemployment", "Random Forest Unemployment"]]
uY = dfs[["Unemployment actual"]]

uX = uX.dropna()
uY = uY.dropna()

In [73]:
gX = dfs[["XGBoost RealGDP", "Random Forest RealGDP"]]
gY = dfs[["RealGDP actual"]]

In [74]:
cX = dfs[["XGBoost Core CPI", "Random Forest Core CPI"]]
cY = dfs[["Core CPI actual"]]

cX = cX.dropna()
cY = cY.dropna()

In [75]:
pX = dfs[["XGBoost Core PCE", "Random Forest Core PCE"]]
pY = dfs[["Core PCE actual"]]

pX = pX.dropna()
pY = pY.dropna()

In [76]:
reg = LinearRegression().fit(gX, gY)
print(reg.score(gX, gY))

pred = reg.predict(gX)
actual = gY

np.sqrt(mean_squared_error(pred, actual))

0.9546869021295435


0.4100763638910511

In [77]:
reg.coef_

array([[0.42815693, 0.57616913]])

In [78]:
reg = LinearRegression().fit(uX, uY)
print(reg.score(uX, uY))

pred = reg.predict(uX)
actual = uY

np.sqrt(mean_squared_error(pred, actual))

0.9442919819963325


0.6702318731501157

In [79]:
reg.coef_

array([[0.74354694, 0.26637404]])

In [80]:
reg = LinearRegression().fit(cX, cY)
print(reg.score(cX, cY))

pred = reg.predict(cX)
actual = cY

np.sqrt(mean_squared_error(pred, actual))

0.922610484523368


0.337232509244397

In [81]:
reg.coef_

array([[-0.45484695,  1.50678078]])

In [82]:
reg = LinearRegression().fit(pX, pY)
print(reg.score(pX, pY))

pred = reg.predict(pX)
actual = pY

np.sqrt(mean_squared_error(pred, actual))

0.9039947016794517


0.37225918993422225

In [83]:
reg.coef_

array([[0.41344445, 0.59470852]])